# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 2)

## 1. Introduction

Being the son of migrant workers having to relocate was a constant through my childhood and even now when trying to find employment. For most of the moves we did growing up there seemed to be a relative or family friend living in our next destination. This provided me and my family a sense of camaraderie, a sense of belonging even as we travel and joined new communities. As such to make it easier to other Latinos to find a place of belonging this project aims to cluster and segment Kansas City's neighborhoods to determine potential locations to live or enjoy a night out.

Hispanic migration to the Kansas City area dates as far back as the 1830s, when the Santa Fe Trail opened up trade from Mexico. But the first major wave of immigration came with the establishment of railroads in Kansas City, Kan., and Kansas City, Mo. In the past 20 years, turmoil in Central America and Mexico has brought a new wave of immigration to the Kansas City area, revitalizing the historic Hispanic communities.

Target audiences for this project does not limit to Latino and Hispanic people but everyone. People could simply decide to look for a restaurant all the time because they are enjoy the food. 


## 2. Data

### Data Required to resolve the problem

In order to make a good choice in finding a location to reside in the Kansas City area the following data is required:
1. List/Information on neighborhoods from Kansas City with their respective Geodata (latitude and longitude)
2. Listed apartments for rent in Kansas City area
4. Venues and amenities in the Kansas City neighborhoods (e.g. top 10) 

This data will provide a set of parameters to determine the location of Latino centric communities where anyone can relocate. Determining the venues in these neighborhoods will allow others who only seek for a place to eat or enjoy a night out with information to make their decision.

## 3. Methodology

After acquiring a data set the CSV file was formated to include only the data of importance to determine the neighborhoods of interest. This was done initially by uploading the dataset, this was then read using the pandas library and converted onto a data-frame for further analysis and manipulation. This was followed by establishing a map with an overlay of all the neighborhoods in the list using the Geodata and Nominatim. To determine potential neighborhoods to live location data using the foursquare API was used to determine the most popular venues in the neighborhoods. These were then listed and sorted determining the most popular establishments per neighborhood. Once this was done a further evaluation of the neighborhoods which had in their top ten venues associated with Hispanic/Latin American culture were evaluated to determine potential rental listings. This was done by clustering the neighborhoods and determining which clusters are potentially more relevant to our search. An assumption was made to establish a cut off and limit the rental search using the most common venue as an indicator of community makeup. With this in mind a data-frame with the neighborhoods with Mexican restaurant as their most common venue was established.

## 4. Results

In this analysis we have explored the locations of Latin or Hispanic centric communities by assuming the most common venue establishes the majority of the make up of the area. Using clustering based on venues within walking distance (defined as 400m radius) we have clustered the neighborhoods into five groups. Of these groups, one has more than one instance of Hispanic communities staples like restaurants and and other amenities that imply the make-up of the community.

The largest clusters share a number of common features. Mexican restaurant, South American venues, as well as bakeries food trucks and grocery stores.

Being specific about the type of each area beyond these shared features is difficult but we note a few differences using the top ten venue types for each cluster:
These clusters showed a varied make up, the one which had most in common was cluster 8 where the majority of the determined neighborhoods of interest formed part of.

## 5. Discussion

Based on our results we have identified five neighborhoods where as per our assumption appeared to be Hispanic or Latin centric communities.

In theory, using these results, Mexican Restaurant could characterize other areas where Latin American or Mexican immigrants might consider for a potential area to rent or enjoy a night out eating and enjoying the community.

This data exploration is not significantly strong secondary correlations with other data sets should be employed to establish a strong correlation between the assumptions and the findings of this exercise.

## Code for the assignment

### Initially to begin this we need to imports the libraries that could potentially be used during the analysis of the data set

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!pip install geopy #installed to use Nominatim and its features to map the data

from geopy.geocoders import Nominatim

!pip install bs4 #initially installed to parse through and scrape the data of wiki, the data was incomplete and it was not used

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # for the clustering of data

import folium
!pip install geocoder
print('Libraries imported.')

     |████████████████████████████████| 122kB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 122kB 6.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 102kB 10.7MB/s ta 0:00:01
     |████████████████████████████████| 92kB 15.9MB/s eta 0:00:01
     |████████████████████████████████| 829kB 9.4MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Libraries imported.


## Mapping Kansas City

In [4]:
## loading the data from csv file after repeated attempts to parse through wiki
Kansas_City_df = pd.read_csv (r'KCMO_neighborhoods.csv') #data set found at data.kcmo.org
Kansas_City_df.rename(columns={'GROUPTITLE': 'Neighborhood'}, inplace=True)
#data had identifiers that did not serve for the purpose of the analysis
KC_df=Kansas_City_df.drop(columns=['ORGANIZATIONID', 'DATECHANGE','STATUS','GROUPID','GROUPWEBSITE','GROUPDESC', 'BOUNDN','BOUNDS','BOUNDW','BOUNDE','SHAPE.AREA','SHAPE.LEN','GROUPTYPE'])
#check the top 5 with the neighborhood and geodata dataframe
KC_df.head()

,Neighborhood,Longitude,Latitude
0,Brittany Oaks Homes Association,-94.678455,39.243321
1,Foxcroft Homes Association,-94.604972,38.924482
2,Lea Manor Homes Association,-94.604405,38.950180
3,Glen Arbor Estates Homes Association,-94.596021,38.921821
4,Red Bridge Hills Homes Association,-94.585355,38.934943


In [5]:
## Locating KC for vizualization of the city
address = 'Kansas City, MO'

geolocator = Nominatim(user_agent="kc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kansas City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kansas City are 39.100105, -94.5781416.


In [6]:
# create map of Kansas using latitude and longitude values
map_KC = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(KC_df['Latitude'], KC_df['Longitude'], KC_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_KC)  
    
map_KC

In [10]:
#define foursquare credentials for determining popular venues around each 
#neighborhood
CLIENT_ID = 'Removed' # your Foursquare ID
CLIENT_SECRET = 'Removed' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XSN5L32WC1XQZF3O0VJF1HR2MZWDC1BM4CSSGQIKTBTBOT51
CLIENT_SECRET:KUFYEBICKHCE25ORSEB3FAJMWWWT5FUXHIZBXX4X14E51DQI


In [11]:
#define a new function to obtain nearby venues  
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
# run the previously defined function through each neighborhood
KC_venues = getNearbyVenues(names=KC_df['Neighborhood'],
                                   latitudes=KC_df['Latitude'],
                                   longitudes=KC_df['Longitude']
                                  )

Brittany Oaks Homes Association
Foxcroft Homes Association
Lea Manor Homes Association
Glen Arbor Estates Homes Association
Red Bridge Hills Homes Association
Red Bud Estates Neighborhood Association
Plaza Westport Neighborhood Association
South Plaza Neighborhood Association
Sacred Heart Homes Association (Westside)
Westside Neighborhood Association
Valentine Neighborhood Association
Westside Planning Committee
Green Meadows Homes Association
Hickory Hollow Homes Association
Coventry-Baughman Neighborhood Association
Platte Brooke Homes Association
Barry Harbor/Hunter's Ridge Homes Association
Loma Vista West Townhouses
Original Loma Vista Neighborhood Association
Rockhill Ridge Neighborhood Association
Rockhill Crest Neighborhood Association
Manheim Park Neighborhood Association
Neighbors United for Action
Blue Hills Neighborhood Association
East Swope Highlands Neighborhood Association
Beacon Hill - McFeders Community Council (formerly Beacon Hill - McFeders Neighborhood Association

In [13]:
# one hot encoding
KC_onehot = pd.get_dummies(KC_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
KC_onehot['Neighborhood'] = KC_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [KC_onehot.columns[-1]] + list(KC_onehot.columns[:-1])
KC_onehot = KC_onehot[fixed_columns]

KC_onehot.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Animal Shelter,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Heliport,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roller Rink,Roof Deck,Rugby Pitch,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Foxcroft Homes Association,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [14]:
# performed ananalysis to determine the venues that are more frequent per neighborhood
KC_grouped = KC_onehot.groupby('Neighborhood').mean().reset_index()
KC_grouped

num_top_venues = 5

for hood in KC_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = KC_grouped[KC_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----40-46 North Neighborhood & Crime Watch----
                  venue  freq
0                  Pool  0.67
1  Gym / Fitness Center  0.33
2            Nail Salon  0.00
3       Other Nightlife  0.00
4  Other Great Outdoors  0.00


----41/42 Montgall Homeowners Association----
               venue  freq
0       Liquor Store   1.0
1                ATM   0.0
2  Outdoor Sculpture   0.0
3      Movie Theater   0.0
4      Moving Target   0.0


----433 Ward Parkway Condominium Association----
                     venue  freq
0      American Restaurant  0.11
1           Clothing Store  0.08
2            Women's Store  0.06
3  New American Restaurant  0.04
4                    Hotel  0.04


----49/63 Coalition----
                      venue  freq
0            Discount Store   0.2
1                Restaurant   0.2
2                       Bar   0.2
3   Health & Beauty Service   0.2
4  College Baseball Diamond   0.2


----Amber Meadows/Amber Lakes Subdivisions----
               venue  freq
0       

## Analysis of the data

In [15]:
#sort the data to determine the most common venues per neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = KC_grouped['Neighborhood']

for ind in np.arange(KC_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(KC_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,40-46 North Neighborhood & Crime Watch,Pool,Gym / Fitness Center,Fondue Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food
1,41/42 Montgall Homeowners Association,Liquor Store,Yoga Studio,Farm,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
2,433 Ward Parkway Condominium Association,American Restaurant,Clothing Store,Women's Store,Seafood Restaurant,New American Restaurant,Hotel,Cosmetics Shop,Coffee Shop,Sporting Goods Shop,Furniture / Home Store
3,49/63 Coalition,Restaurant,Discount Store,College Baseball Diamond,Health & Beauty Service,Bar,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
4,Amber Meadows/Amber Lakes Subdivisions,Playground,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop


In [16]:
# set number of clusters
kclusters = 10

KC_grouped_clustering = KC_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(KC_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

KC_merged = KC_df

# merge KC_grouped with KC_df to add latitude/longitude for each neighborhood
KC_merged = KC_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

KC_merged.head() # check the last columns!


,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brittany Oaks Homes Association,-94.678455,39.243321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Foxcroft Homes Association,-94.604972,38.924482,6.0,BBQ Joint,Electronics Store,Yoga Studio,Fondue Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Farm
2,Lea Manor Homes Association,-94.604405,38.950180,5.0,Pool,Construction & Landscaping,Trail,Yoga Studio,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop
3,Glen Arbor Estates Homes Association,-94.596021,38.921821,6.0,Café,Electronics Store,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
4,Red Bridge Hills Homes Association,-94.585355,38.934943,6.0,Hotel,Breakfast Spot,Coffee Shop,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant


In [17]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(KC_merged['Latitude'], KC_merged['Longitude'], KC_merged['Neighborhood'], KC_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[-1 ],
        fill=True,
        fill_color=rainbow[-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster analysis for areas of interest

Each cluster was analyzed to determine the areas of interest by keywords on the most popular venues. The most common being Mexican restaurant, followed by food truck, and South American restaurant. 

In [18]:
KC_merged.loc[KC_merged['Cluster Labels'] == 0, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Loma Vista West Townhouses,Pizza Place,Sandwich Place,Chinese Restaurant,Spa,Asian Restaurant,Discount Store,American Restaurant,Grocery Store,Food Service
30,Green Haven Neighborhood Association,Comic Shop,Women's Store,American Restaurant,Pizza Place,Department Store,Pet Store,Cupcake Shop,Auto Garage,Automotive Shop
33,Wedgewood Pointe Neighborhood Association,Farm,Frame Store,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
47,Walnut Grove Neighborhood Association,Bus Station,Garden Center,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant
49,Southern Hills Subdivision & Annex Homes Assoc...,Coffee Shop,Rock Club,Yoga Studio,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop
50,Birchwood Hills Homes Association,Performing Arts Venue,Discount Store,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Yoga Studio
54,Westchester Homes Association,Spa,Grocery Store,Park,Sandwich Place,Fried Chicken Joint,Chinese Restaurant,Pizza Place,Discount Store,American Restaurant
56,Palestine Neighborhood Development Corporation...,Ice Cream Shop,Home Service,Discount Store,Convenience Store,Dry Cleaner,Lawyer,Fried Chicken Joint,Hobby Shop,Event Space
57,Swope Parkway/Elmwood Neighborhood Association...,Yoga Studio,Farm,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
85,Oak Park Neighborhood Association,Ice Cream Shop,Lawyer,Discount Store,Dry Cleaner,Fried Chicken Joint,Hookah Bar,Home Service,Food Truck,Food Service


In [19]:
KC_merged.loc[KC_merged['Cluster Labels'] == 1, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
110,Foxtown West Neighborhood Association,Construction & Landscaping,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food Court
120,Ravenwood Summerset Neighborhood Association,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
122,Woodridge Neighborhood Association,Construction & Landscaping,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
186,Little Blue Valley Homes Association,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
188,Timber Valley Neighborhood Association (conti...,Construction & Landscaping,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food Court


In [20]:
KC_merged.loc[KC_merged['Cluster Labels'] == 2, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Blue Hills Neighborhood Association,Yoga Studio,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Factory
27,Boston Heights and Mount Hope Neighborhood Ass...,Park,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
42,Armour Hills Homes Association,Spa,Playground,Trail,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Fondue Restaurant
58,Sunset Hill Neighborhood Association,Park,Athletics & Sports,Yoga Studio,Fondue Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food
89,Pendleton Heights Neighborhood Association,Disc Golf,Thrift / Vintage Store,Construction & Landscaping,Yoga Studio,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
114,Scarritt Renaissance Neighborhood Association,Gym,Taco Place,Convenience Store,Museum,Yoga Studio,Flower Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market
115,Blue Valley Neighborhood Association,Park,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
117,Little Village Homes Association,Park,Garden,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food
123,Highland Acres-Lakeside Heights Homes Association,Park,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Yoga Studio,Farm
153,Sheffield Neighborhood Association,Park,Taco Place,Yoga Studio,Flower Shop,Farmers Market,Fish & Chips Shop,Flea Market,Fondue Restaurant,Factory


In [21]:
KC_merged.loc[KC_merged['Cluster Labels'] == 3, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,Bridgepointe Homes Association,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
128,Santa Fe Hills Homes Association,Trail,Yoga Studio,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food
237,Mid Oaks Neighborhood Association (Prather),American Restaurant,Food & Drink Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Yoga Studio
246,Parkway Village HOA,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop


In [22]:
KC_merged.loc[KC_merged['Cluster Labels'] == 4, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
133,Park Plaza Home Owners Association,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
189,Amber Meadows/Amber Lakes Subdivisions,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
210,Golden Oaks Neighborhood Association,Food Court,Yoga Studio,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop


In [23]:
KC_merged.loc[KC_merged['Cluster Labels'] == 5, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Lea Manor Homes Association,Construction & Landscaping,Trail,Yoga Studio,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop
46,The Coves Homes Association,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
59,40-46 North Neighborhood & Crime Watch,Gym / Fitness Center,Fondue Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food
61,Carriage Hill Estates Homes Association,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
126,Woodson Estates Homes Association,Pool,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Yoga Studio
137,Kirktown Homes Association,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
138,Kirkside Homes Association,Home Service,Yoga Studio,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop
155,Sherwood Estates Homes Association,Art Gallery,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food Court
172,Walden Homeowners Association,Furniture / Home Store,Playground,Yoga Studio,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop
183,Communities of Northbrook,Bar,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop


In [24]:
KC_merged.loc[KC_merged['Cluster Labels'] == 6, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Foxcroft Homes Association,Electronics Store,Yoga Studio,Fondue Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Farm
3,Glen Arbor Estates Homes Association,Electronics Store,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
4,Red Bridge Hills Homes Association,Breakfast Spot,Coffee Shop,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant
5,Red Bud Estates Neighborhood Association,Lighthouse,Restaurant,Sporting Goods Shop,Miscellaneous Shop,Home Service,Fabric Shop,Food Truck,Food Service,Food Court
6,Plaza Westport Neighborhood Association,Hotel,Arts & Crafts Store,Pizza Place,Spa,Cocktail Bar,New American Restaurant,Salon / Barbershop,Comedy Club,Thai Restaurant
7,South Plaza Neighborhood Association,American Restaurant,Sandwich Place,BBQ Joint,Dive Bar,Park,Electronics Store,Coffee Shop,Mexican Restaurant,Breakfast Spot
8,Sacred Heart Homes Association (Westside),BBQ Joint,Office,Nightclub,Coffee Shop,Brewery,American Restaurant,Football Stadium,Food Truck,Food Service
9,Westside Neighborhood Association,Art Gallery,Theater,Restaurant,Park,South American Restaurant,Dive Bar,Distillery,Fountain,Football Stadium
10,Valentine Neighborhood Association,Ramen Restaurant,Massage Studio,Chinese Restaurant,Gay Bar,Coffee Shop,Gift Shop,Sports Bar,Fast Food Restaurant,Ethiopian Restaurant
11,Westside Planning Committee,Brewery,Taco Place,Theater,Park,Office,Restaurant,New American Restaurant,Factory,Fountain


In [25]:
KC_merged.loc[KC_merged['Cluster Labels'] == 7, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
146,Palmer Drive Block Club,Yoga Studio,Farm,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
243,Center Planning and Development Council (CPDC),Yoga Studio,Farm,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


## Cluster 9 (labeled 8) 

This cluster was found to be disproportionately larger than the others, due to this it seemed to have captured the majority of the areas of interest. 

In [26]:
KC_merged.loc[KC_merged['Cluster Labels'] == 8, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
159,Coachlight Square Homes Association,Yoga Studio,Farm,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [27]:
KC_merged.loc[KC_merged['Cluster Labels'] == 9, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
112,Candlelight Ridge Homeowners Association,Beach,Yoga Studio,Farm,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
125,White Oaks West Neighborhood Association (form...,Yoga Studio,Farm,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
145,Park Farms Neighborhood Watch,Beach,Food,Yoga Studio,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Fried Chicken Joint
200,Lakeview Gardens Neighborhood Association,Yoga Studio,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Fondue Restaurant


In [28]:
## New Data set that was used to map the potential areas found with our assumptions
KC_ltnx= KC_merged.loc[(KC_merged['1st Most Common Venue']==('Mexican Restaurant'))]
KC_ltnx

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Sacred Heart Homes Association (Westside),-94.600631,39.076984,6.0,Mexican Restaurant,BBQ Joint,Office,Nightclub,Coffee Shop,Brewery,American Restaurant,Football Stadium,Food Truck,Food Service
9,Westside Neighborhood Association,-94.600244,39.086082,6.0,Mexican Restaurant,Art Gallery,Theater,Restaurant,Park,South American Restaurant,Dive Bar,Distillery,Fountain,Football Stadium
11,Westside Planning Committee,-94.597936,39.085088,6.0,Mexican Restaurant,Brewery,Taco Place,Theater,Park,Office,Restaurant,New American Restaurant,Factory,Fountain
208,Indian Mound Neighborhood Association,-94.520484,39.111942,6.0,Mexican Restaurant,Grocery Store,Park,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food
253,Morningside Neighborhood Association,-94.588139,39.015244,6.0,Mexican Restaurant,Grocery Store,American Restaurant,Bakery,Ice Cream Shop,Coffee Shop,Italian Restaurant,Burger Joint,Furniture / Home Store,Fried Chicken Joint


In [32]:
# create map of Kansas using latitude and longitude values
map_KC_ltnx = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, neighborhood in zip(KC_ltnx['Latitude'], KC_ltnx['Longitude'], KC_ltnx['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_KC_ltnx)  
    
map_KC_ltnx

## Focusing on Indian Mound Neighborhood Association

This area is in Independence Avenue is historically hispanic, due to this we will map the area further and map the venues nearby this location.

In [39]:
search_query = 'Mexican'
radius = 2000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results2 = requests.get(url).json()
results2
# assign relevant part of JSON to venues
IMNA_venues = results2['response']['venues']

Mexican .... OK!


In [40]:
# tranform venues into a dataframe
IMNA_df = json_normalize(IMNA_venues)

IMNA_df

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4ad4c01ef964a520bcf120e3,Manny's Mexican Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1592572950,False,207 Southwest Blvd,at Wyandotte St,39.089144,-94.586654,"[{'label': 'display', 'lat': 39.08914444282776...",1424,64108,US,Kansas City,MO,United States,"[207 Southwest Blvd (at Wyandotte St), Kansas ...",33288885,NaN,NaN,NaN,NaN,NaN,NaN
1,4b5ddb22f964a520267029e3,Z-Teca Mexican Grill,"[{'id': '4bf58dd8d48988d153941735', 'name': 'B...",v-1592572950,False,2450 Grand Blvd,NaN,39.082651,-94.581684,"[{'label': 'display', 'lat': 39.08265085904718...",1966,64108,US,Kansas City,MO,United States,"[2450 Grand Blvd, Kansas City, MO 64108, Unite...",NaN,1633074,https://www.grubhub.com/restaurant/z-teca-fres...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png
2,4af47759f964a520c8f221e3,Teocali Mexican Restaurant & Cantina,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1592572950,False,2512 Holmes St,btwn E 25th & 26th St,39.081016,-94.576175,"[{'label': 'display', 'lat': 39.08101597613341...",2131,64108,US,Kansas City,MO,United States,"[2512 Holmes St (btwn E 25th & 26th St), Kansa...",NaN,1728916,https://www.grubhub.com/restaurant/teocali-251...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png
3,4adf2783f964a520e67721e3,Chipotle Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1592572950,False,1370 Walnut St,at E 14th St,39.097368,-94.582474,"[{'label': 'display', 'lat': 39.09736788816193...",482,64106,US,Kansas City,MO,United States,"[1370 Walnut St (at E 14th St), Kansas City, M...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4e0e690818a8bf9784cd3abf,Consulate General of Mexico,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1592572950,False,1600 Baltimore Ave,16th St.,39.093947,-94.584256,"[{'label': 'display', 'lat': 39.093947, 'lng':...",865,64108,US,Kansas City,MO,United States,"[1600 Baltimore Ave (16th St.), Kansas City, M...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4b675b23f964a520054a2be3,La Fonda El Taquito Mexican Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1592572950,False,800 Southwest Blvd,btw Summit and SW Blvd.,39.084922,-94.593982,"[{'label': 'display', 'lat': 39.08492210037836...",2174,64108,US,Kansas City,MO,United States,"[800 Southwest Blvd (btw Summit and SW Blvd.),...",NaN,1605317,https://www.grubhub.com/restaurant/la-fonda-el...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png
6,4c79c72fa86837042ff5114d,La Mexicana,[],v-1592572950,False,NaN,NaN,39.085278,-94.584722,"[{'label': 'display', 'lat': 39.08527777777778...",1745,64108,US,Kansas City,MO,United States,"[Kansas City, MO 64108, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in IMNA_df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = IMNA_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Manny's Mexican Restaurant,Mexican Restaurant,207 Southwest Blvd,at Wyandotte St,39.089144,-94.586654,"[{'label': 'display', 'lat': 39.08914444282776...",1424,64108,US,Kansas City,MO,United States,"[207 Southwest Blvd (at Wyandotte St), Kansas ...",4ad4c01ef964a520bcf120e3
1,Z-Teca Mexican Grill,Burrito Place,2450 Grand Blvd,NaN,39.082651,-94.581684,"[{'label': 'display', 'lat': 39.08265085904718...",1966,64108,US,Kansas City,MO,United States,"[2450 Grand Blvd, Kansas City, MO 64108, Unite...",4b5ddb22f964a520267029e3
2,Teocali Mexican Restaurant & Cantina,Mexican Restaurant,2512 Holmes St,btwn E 25th & 26th St,39.081016,-94.576175,"[{'label': 'display', 'lat': 39.08101597613341...",2131,64108,US,Kansas City,MO,United States,"[2512 Holmes St (btwn E 25th & 26th St), Kansa...",4af47759f964a520c8f221e3
3,Chipotle Mexican Grill,Mexican Restaurant,1370 Walnut St,at E 14th St,39.097368,-94.582474,"[{'label': 'display', 'lat': 39.09736788816193...",482,64106,US,Kansas City,MO,United States,"[1370 Walnut St (at E 14th St), Kansas City, M...",4adf2783f964a520e67721e3
4,Consulate General of Mexico,Embassy / Consulate,1600 Baltimore Ave,16th St.,39.093947,-94.584256,"[{'label': 'display', 'lat': 39.093947, 'lng':...",865,64108,US,Kansas City,MO,United States,"[1600 Baltimore Ave (16th St.), Kansas City, M...",4e0e690818a8bf9784cd3abf
5,La Fonda El Taquito Mexican Restaurant,Mexican Restaurant,800 Southwest Blvd,btw Summit and SW Blvd.,39.084922,-94.593982,"[{'label': 'display', 'lat': 39.08492210037836...",2174,64108,US,Kansas City,MO,United States,"[800 Southwest Blvd (btw Summit and SW Blvd.),...",4b675b23f964a520054a2be3
6,La Mexicana,None,NaN,NaN,39.085278,-94.584722,"[{'label': 'display', 'lat': 39.08527777777778...",1745,64108,US,Kansas City,MO,United States,"[Kansas City, MO 64108, United States]",4c79c72fa86837042ff5114d


In [44]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Indian Mound Neighborhood Association',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Mexican business as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map